In [1]:
# Install Flask and Flask-Cors for API development
!pip install flask flask-cors flask-ngrok


In [2]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from flask_cors import CORS

app = Flask(__name__)
run_with_ngrok(app)
CORS(app)

# In-memory data structures
users = {}  # Format: {username: {"password": "password", "posts": []}}
posts = []  # List of dictionaries containing post details

# Route to register a new user
@app.route('/register', methods=['POST'])
def register():
    data = request.json
    username = data.get('username')
    password = data.get('password')
    if username in users:
        return jsonify({"message": "User already exists"}), 400
    users[username] = {"password": password, "posts": []}
    return jsonify({"message": "User registered successfully"}), 200

# Route to login
@app.route('/login', methods=['POST'])
def login():
    data = request.json
    username = data.get('username')
    password = data.get('password')
    user = users.get(username)
    if user and user['password'] == password:
        return jsonify({"message": "Login successful"}), 200
    return jsonify({"message": "Invalid credentials"}), 401

# Route to add a new post
@app.route('/add_post', methods=['POST'])
def add_post():
    data = request.json
    username = data.get('username')
    content = data.get('content')
    if username not in users:
        return jsonify({"message": "User not found"}), 404
    post = {"username": username, "content": content}
    posts.append(post)
    users[username]["posts"].append(post)
    return jsonify({"message": "Post added successfully"}), 200

# Route to get all posts
@app.route('/get_posts', methods=['GET'])
def get_posts():
    return jsonify({"posts": posts}), 200

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 199, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

In [31]:
%%writefile index.html
<!DOCTYPE html>
<html>
<head>
    <title>Instagram Clone</title>
    <style>
        .post {
            margin: 10px;
            border: 1px solid #ddd;
            padding: 10px;
        }
    </style>
</head>
<body>
    <h1>Instagram Clone</h1>
    <div id="login">
        <h2>Login</h2>
        <input type="text" id="username" placeholder="Username">
        <input type="password" id="password" placeholder="Password">
        <button onclick="login()">Login</button>
        <p id="login-message"></p>
    </div>
    <div id="post-section" style="display: none;">
        <h2>New Post</h2>
        <textarea id="post-content" placeholder="What's on your mind?"></textarea>
        <button onclick="addPost()">Post</button>
        <h2>Feed</h2>
        <div id="feed"></div>
    </div>

    <script>
        const apiUrl = 'http://localhost:5000'; // Change this to the ngrok URL when running in Colab

        function login() {
            const username = document.getElementById('username').value;
            const password = document.getElementById('password').value;

            fetch(`${apiUrl}/login`, {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ username, password })
            })
            .then(response => response.json())
            .then(data => {
                if (data.message === 'Login successful') {
                    document.getElementById('login').style.display = 'none';
                    document.getElementById('post-section').style.display = 'block';
                    loadPosts();
                } else {
                    document.getElementById('login-message').innerText = data.message;
                }
            });
        }

        function addPost() {
            const content = document.getElementById('post-content').value;

            fetch(`${apiUrl}/add_post`, {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ username: 'testUser', content }) // Replace with dynamic username after login
            })
            .then(response => response.json())
            .then(() => loadPosts());
        }

        function loadPosts() {
            fetch(`${apiUrl}/get_posts`)
            .then(response => response.json())
            .then(data => {
                const feed = document.getElementById('feed');
                feed.innerHTML = data.posts.map(post => `<div class="post"><b>${post.username}</b>: ${post.content}</div>`).join('');
            });
        }
    </script>
</body>
</html>


Writing index.html


In [32]:
!python -m http.server 8000


Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...

Keyboard interrupt received, exiting.
